# 1. Data Extraction via API (2021 - 2023) 
Note: Following script does not use hopworks

In [7]:
import requests
from datetime import datetime, timedelta
import pandas as pd

# API endpoint for retrieving PM2.5 information
api_url = "https://api.data.gov.sg/v1/environment/pm25"

# Define the start and end timestamps
start_timestamp = datetime(2021, 11, 30, 0, 0, 0)  # Take 30th November 2021, 1200 as start date
end_timestamp = datetime(2023, 12, 1, 0, 0, 0)    # Take 1st Dec 2023, 0000 as end date

# Define the time interval (e.g., hourly)
time_interval = timedelta(hours=1)            # Using 1 hourly interval

# Initialize an empty list to store the data
all_data = []

# Make API requests for each timestamp in the specified range
current_timestamp = start_timestamp
while current_timestamp <= end_timestamp:
    # Format the timestamp in the required format
    formatted_timestamp = current_timestamp.strftime("%Y-%m-%dT%H:%M:%S%z")

    # Make the API request
    response = requests.get(api_url, params={"date_time": formatted_timestamp})

    # Process the response (you can adapt this based on the actual API response format)
    if response.status_code == 200:
        data = response.json()  # Assuming the API returns JSON
        # Extract relevant information and append to the list
        timestamp_data = {
            "timestamp": formatted_timestamp,
            "update_timestamp": data.get("items", [])[0].get("update_timestamp", ""),
            "pm25_readings": data.get("items", [])[0].get("readings", {}).get("pm25_one_hourly", {})
        }
        all_data.append(timestamp_data)
    else:
        print(f"Error for timestamp {formatted_timestamp}: {response.status_code}, {response.text}")

    # Move to the next timestamp
    current_timestamp += time_interval

# Convert the accumulated data into a DataFrame
df = pd.DataFrame(all_data)

# Normalize the 'pm25_readings' column
pm25_normalized = pd.json_normalize(df['pm25_readings'])

# Concatenate the normalized columns with the original DataFrame
df = pd.concat([df, pm25_normalized], axis=1)

# Drop the original 'pm25_readings' column
df = df.drop(['pm25_readings'], axis=1)

# Optionally, you can drop unnecessary columns or reorder them
df = df[['timestamp', 'update_timestamp', 'west', 'east', 'central', 'south', 'north']]


# Export the DataFrame to a CSV file
df.to_csv('pm25_data_raw.csv', index=False)

# Print or use the resulting DataFrame as needed
print(df)


                 timestamp           update_timestamp  west  east  central  \
0      2021-11-30T00:00:00  2021-11-30T00:08:52+08:00   5.0  12.0     14.0   
1      2021-11-30T01:00:00  2021-11-30T01:08:52+08:00   8.0  14.0     12.0   
2      2021-11-30T02:00:00  2021-11-30T02:08:52+08:00   9.0  17.0     14.0   
3      2021-11-30T03:00:00  2021-11-30T03:08:52+08:00   6.0  14.0     18.0   
4      2021-11-30T04:00:00  2021-11-30T04:08:52+08:00   6.0  14.0     17.0   
...                    ...                        ...   ...   ...      ...   
17540  2023-11-30T20:00:00  2023-11-30T20:03:53+08:00  14.0  10.0      9.0   
17541  2023-11-30T21:00:00  2023-11-30T21:03:52+08:00   3.0   9.0      3.0   
17542  2023-11-30T22:00:00  2023-11-30T22:03:53+08:00   5.0   9.0      8.0   
17543  2023-11-30T23:00:00  2023-11-30T23:03:52+08:00   5.0   9.0      9.0   
17544  2023-12-01T00:00:00  2023-12-01T00:03:52+08:00   4.0   5.0      5.0   

       south  north  
0        6.0   14.0  
1        9.0   13.0

# 2. Data Cleaning

In [75]:
# To save time from reloading above API - load from previous csv file saved (if already loaded, you may skip this step)
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('pm25_data_raw.csv')

# Display the DataFrame
print(df)

                 timestamp           update_timestamp  west  east  central  \
0      2021-11-30T00:00:00  2021-11-30T00:08:52+08:00   5.0  12.0     14.0   
1      2021-11-30T01:00:00  2021-11-30T01:08:52+08:00   8.0  14.0     12.0   
2      2021-11-30T02:00:00  2021-11-30T02:08:52+08:00   9.0  17.0     14.0   
3      2021-11-30T03:00:00  2021-11-30T03:08:52+08:00   6.0  14.0     18.0   
4      2021-11-30T04:00:00  2021-11-30T04:08:52+08:00   6.0  14.0     17.0   
...                    ...                        ...   ...   ...      ...   
17540  2023-11-30T20:00:00  2023-11-30T20:03:53+08:00  14.0  10.0      9.0   
17541  2023-11-30T21:00:00  2023-11-30T21:03:52+08:00   3.0   9.0      3.0   
17542  2023-11-30T22:00:00  2023-11-30T22:03:53+08:00   5.0   9.0      8.0   
17543  2023-11-30T23:00:00  2023-11-30T23:03:52+08:00   5.0   9.0      9.0   
17544  2023-12-01T00:00:00  2023-12-01T00:03:52+08:00   4.0   5.0      5.0   

       south  north  
0        6.0   14.0  
1        9.0   13.0

# Find missing data 

In [76]:
import pandas as pd

# Count the missing values in df and test datasets
df_missing = df.isnull().sum().sort_values(ascending=False)[:13].reset_index()

# Calculate percentage of missing values
df_missing_percentage = (df_missing[0] / len(df)) * 100

# Add columns for missing values and percentages
df_missing['df_missing_percentage'] = df_missing_percentage

# Rename columns for clarity
df_missing.columns = ['columns', 'df_missing', 'df_missing_percentage']

df_missing 

,columns,df_missing,df_missing_percentage
0,update_timestamp,418,2.382445
1,west,418,2.382445
2,east,418,2.382445
3,central,418,2.382445
4,south,418,2.382445
5,north,418,2.382445
6,timestamp,0,0.000000


In [77]:
# Check for missing values in df
missing_values_df = df[df.isnull().any(axis=1)]

# Print the DataFrame with rows containing missing values
print("Rows with any missing values:")
print(missing_values_df)

Rows with any missing values:
                 timestamp update_timestamp  west  east  central  south  north
8384   2022-11-14T08:00:00              NaN   NaN   NaN      NaN    NaN    NaN
8385   2022-11-14T09:00:00              NaN   NaN   NaN      NaN    NaN    NaN
8386   2022-11-14T10:00:00              NaN   NaN   NaN      NaN    NaN    NaN
8387   2022-11-14T11:00:00              NaN   NaN   NaN      NaN    NaN    NaN
8388   2022-11-14T12:00:00              NaN   NaN   NaN      NaN    NaN    NaN
...                    ...              ...   ...   ...      ...    ...    ...
16950  2023-11-06T06:00:00              NaN   NaN   NaN      NaN    NaN    NaN
16951  2023-11-06T07:00:00              NaN   NaN   NaN      NaN    NaN    NaN
16952  2023-11-06T08:00:00              NaN   NaN   NaN      NaN    NaN    NaN
16953  2023-11-06T09:00:00              NaN   NaN   NaN      NaN    NaN    NaN
16954  2023-11-06T10:00:00              NaN   NaN   NaN      NaN    NaN    NaN

[418 rows x 7 columns

In [78]:
# Drop rows with missing values from df
df_cleaned = df.dropna()

# Print the DataFrame without missing values
print("DataFrame without missing values:")
print(df_without_missing)

DataFrame without missing values:
                 timestamp           update_timestamp  west  east  central  \
0      2021-11-30T00:00:00  2021-11-30T00:08:52+08:00   5.0  12.0     14.0   
1      2021-11-30T01:00:00  2021-11-30T01:08:52+08:00   8.0  14.0     12.0   
2      2021-11-30T02:00:00  2021-11-30T02:08:52+08:00   9.0  17.0     14.0   
3      2021-11-30T03:00:00  2021-11-30T03:08:52+08:00   6.0  14.0     18.0   
4      2021-11-30T04:00:00  2021-11-30T04:08:52+08:00   6.0  14.0     17.0   
...                    ...                        ...   ...   ...      ...   
17540  2023-11-30T20:00:00  2023-11-30T20:03:53+08:00  14.0  10.0      9.0   
17541  2023-11-30T21:00:00  2023-11-30T21:03:52+08:00   3.0   9.0      3.0   
17542  2023-11-30T22:00:00  2023-11-30T22:03:53+08:00   5.0   9.0      8.0   
17543  2023-11-30T23:00:00  2023-11-30T23:03:52+08:00   5.0   9.0      9.0   
17544  2023-12-01T00:00:00  2023-12-01T00:03:52+08:00   4.0   5.0      5.0   

       south  north        da

# 3. Feature Engineering

# Feature Construction: timestamp - Date | Time 

In [79]:
#Split 'timestamp' into 2 columns - Date | Time
df_cleaned[['date', 'time']] = df_cleaned['timestamp'].str.split('T', expand=True)
df_cleaned


C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\3530707872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['date', 'time']] = df_cleaned['timestamp'].str.split('T', expand=True)
C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\3530707872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['date', 'time']] = df_cleaned['timestamp'].str.split('T', expand=True)


,timestamp,update_timestamp,west,east,central,south,north,date,time
0,2021-11-30T00:00:00,2021-11-30T00:08:52+08:00,5.0,12.0,14.0,6.0,14.0,2021-11-30,00:00:00
1,2021-11-30T01:00:00,2021-11-30T01:08:52+08:00,8.0,14.0,12.0,9.0,13.0,2021-11-30,01:00:00
2,2021-11-30T02:00:00,2021-11-30T02:08:52+08:00,9.0,17.0,14.0,12.0,16.0,2021-11-30,02:00:00
3,2021-11-30T03:00:00,2021-11-30T03:08:52+08:00,6.0,14.0,18.0,6.0,12.0,2021-11-30,03:00:00
4,2021-11-30T04:00:00,2021-11-30T04:08:52+08:00,6.0,14.0,17.0,7.0,14.0,2021-11-30,04:00:00
...,...,...,...,...,...,...,...,...,...
17540,2023-11-30T20:00:00,2023-11-30T20:03:53+08:00,14.0,10.0,9.0,8.0,5.0,2023-11-30,20:00:00
17541,2023-11-30T21:00:00,2023-11-30T21:03:52+08:00,3.0,9.0,3.0,6.0,4.0,2023-11-30,21:00:00
17542,2023-11-30T22:00:00,2023-11-30T22:03:53+08:00,5.0,9.0,8.0,4.0,4.0,2023-11-30,22:00:00
17543,2023-11-30T23:00:00,2023-11-30T23:03:52+08:00,5.0,9.0,9.0,10.0,5.0,2023-11-30,23:00:00


In [80]:
#Split 'date' into 3 columns - Year | Month | Day
df_cleaned[['year','month','day']] = df_cleaned['date'].str.split('-', expand=True)


C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\2012494841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['year','month','day']] = df_cleaned['date'].str.split('-', expand=True)
C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\2012494841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['year','month','day']] = df_cleaned['date'].str.split('-', expand=True)
C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\2012494841.py:2: SettingWithCopyWarning: 
A value is trying to be se

In [81]:
#split 'time' into only hour 
df_cleaned['hour'] = df_cleaned['time'].str[:2]

C:\Users\xueer\AppData\Local\Temp\ipykernel_26208\2347755829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['hour'] = df_cleaned['time'].str[:2]


In [82]:
# Drop timestamp and updated timestamp only keep necessary columns. 
df_cleaned = df_cleaned.drop(['timestamp', 'update_timestamp','date', 'time'], axis=1)

In [83]:
# Export the DataFrame to a CSV file
df_cleaned.to_csv('pm25_data_cleaned.csv', index=False)
df_cleaned.head()


,west,east,central,south,north,year,month,day,hour
0,5.0,12.0,14.0,6.0,14.0,2021,11,30,00
1,8.0,14.0,12.0,9.0,13.0,2021,11,30,01
2,9.0,17.0,14.0,12.0,16.0,2021,11,30,02
3,6.0,14.0,18.0,6.0,12.0,2021,11,30,03
4,6.0,14.0,17.0,7.0,14.0,2021,11,30,04


# 4. Experiment with Different Models (before hyperparameter Tuning)
A lower MSE indicates better model performance

In [84]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error

# Assuming your data is stored in a variable named 'df_cleaned'
# Extract features and target variables
X = df_cleaned.drop(columns=['west', 'east', 'central', 'south', 'north'])
y = df_cleaned[['west', 'east', 'central', 'south', 'north']]

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'K-Nearest Neighbours': KNeighborsRegressor(),   
    'SVR': SVR(),
    'Linear Regression': LinearRegression(),
    'XGBoost': XGBRegressor(),
    'Random Forest': RandomForestRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Bagging': BaggingRegressor(),
    'Gradient Boosted Trees': GradientBoostingRegressor(),
    'HistGradient Boosted Trees': HistGradientBoostingRegressor()
}

# Evaluate models using k-fold cross-validation and calculate mean squared errors (5-fold)
for model_name, model in models.items():
    # Wrap the model in MultiOutputRegressor for multi-output regression
    multioutput_model = MultiOutputRegressor(model)
    
    # Perform 5-fold cross-validation
    scores = cross_val_score(multioutput_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mean_mse = round(-scores.mean(), 4)  # Convert negative MSE to positive
    
    # Train the model
    multioutput_model.fit(X_train, y_train)
    
    # Predictions on the test set
    y_pred = multioutput_model.predict(X_test)
    
    # Test mean squared error
    test_mse = round(mean_squared_error(y_test, y_pred), 4)
    
    # Print mean squared errors with 4 decimal places
    print(f'{model_name} Cross-Validation Mean Squared Error: {mean_mse:.4f}')
    print(f'{model_name} Test Mean Squared Error: {test_mse:.4f}\n')


K-Nearest Neighbours Cross-Validation Mean Squared Error: 25.4519
K-Nearest Neighbours Test Mean Squared Error: 22.9867

SVR Cross-Validation Mean Squared Error: 40.8408
SVR Test Mean Squared Error: 40.4865

Linear Regression Cross-Validation Mean Squared Error: 47.3943
Linear Regression Test Mean Squared Error: 47.3775

XGBoost Cross-Validation Mean Squared Error: 21.0686
XGBoost Test Mean Squared Error: 19.9004

Random Forest Cross-Validation Mean Squared Error: 16.0689
Random Forest Test Mean Squared Error: 14.1625

AdaBoost Cross-Validation Mean Squared Error: 46.6465
AdaBoost Test Mean Squared Error: 46.9082

Bagging Cross-Validation Mean Squared Error: 17.6467
Bagging Test Mean Squared Error: 15.4516

Gradient Boosted Trees Cross-Validation Mean Squared Error: 33.2426
Gradient Boosted Trees Test Mean Squared Error: 33.2620

HistGradient Boosted Trees Cross-Validation Mean Squared Error: 23.9681
HistGradient Boosted Trees Test Mean Squared Error: 23.6923

